# Super Store Analysis

## Installing dependencies

In [33]:
# !pip install xlrd
# !pip install pandas
# !pip install numpy
# !pip install mysql-connector-python
# !pip install sqlalchemy
# !pip install pymysql

## Setting up the environment

In [34]:
import re
import warnings
import pandas as pd
import numpy as np
import mysql.connector as mysql
from sqlalchemy import create_engine, text

warnings.filterwarnings('ignore')

---

## Import data

In [35]:
file_path = 'data-src/Sample - Superstore.xls'
orders_df = pd.read_excel(file_path, sheet_name=0, header=0)
people_df = pd.read_excel(file_path, sheet_name=1, header=0)
returns_df = pd.read_excel(file_path, sheet_name=2, header=0)

---

## Data exploration

### Orders

**Overview**

In [ ]:
orders_df.head()

In [ ]:
orders_df.tail()

**Details**

- measurement in rows x columns
- columns and datatypes

In [38]:
print('(row, col)')
orders_df.shape

(row, col)


(10194, 21)

In [ ]:
orders_df.info()

---

### People

In [40]:
people_df.head()

,Regional Manager,Region
0,Sadie Pawthorne,West
1,Chuck Magee,East
2,Roxanne Rodriguez,Central
3,Fred Suzuki,South


In [41]:
people_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Regional Manager  4 non-null      object
 1   Region            4 non-null      object
dtypes: object(2)
memory usage: 196.0+ bytes


---

### Returns

**Overview**

In [42]:
returns_df.head()

,Returned,Order ID
0,Yes,US-2021-100762
1,Yes,US-2021-100762
2,Yes,US-2021-100762
3,Yes,US-2021-100762
4,Yes,US-2021-100867


In [43]:
returns_df.tail()

,Returned,Order ID
795,Yes,US-2024-147886
796,Yes,US-2024-147998
797,Yes,US-2024-151127
798,Yes,US-2024-155999
799,Yes,US-2024-155999


#### Details

In [44]:
returns_df.shape

(800, 2)

In [45]:
returns_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Returned  800 non-null    object
 1   Order ID  800 non-null    object
dtypes: object(2)
memory usage: 12.6+ KB


---

## Data cleaning

### Orders

**Duplicate values**

In [46]:
print(orders_df.duplicated().sum())

0


**Null values**

In [47]:
print(orders_df.isnull().sum())

Row ID            0
Order ID          0
Order Date        0
Ship Date         0
Ship Mode         0
Customer ID       0
Customer Name     0
Segment           0
Country/Region    0
City              0
State/Province    0
Postal Code       0
Region            0
Product ID        0
Category          0
Sub-Category      0
Product Name      0
Sales             0
Quantity          0
Discount          0
Profit            0
dtype: int64


---

### Returns

**Duplicate values**

In [48]:
print(returns_df.duplicated().sum())

504


**Remove duplicates**

In [49]:
returns_df.drop_duplicates(inplace=True) # no need to use keep= as we are not dealing with datetime data

**Null values**

In [50]:
returns_df.isnull().sum()

Returned    0
Order ID    0
dtype: int64

---

## Data Transformation

### **Reformat headers**

In [51]:
def snake_case(data:str) -> str:
	return(data.lower().strip().replace(' ','_').replace('-', '_').replace('/', '_'))

### Orders

In [52]:
orders_df.columns = [snake_case(col) for col in orders_df.columns]
orders_df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country_region,city,...,postal_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
0,1,US-2021-103800,2021-01-03,2021-01-07,Standard Class,DP-13000,Darren Powers,Consumer,United States,Houston,...,77095,Central,OFF-PA-10000174,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.448,2,0.2,5.5512
1,2,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-BI-10004094,Office Supplies,Binders,GBC Standard Plastic Binding Systems Combs,3.540,2,0.8,-5.4870
2,3,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-LA-10003223,Office Supplies,Labels,Avery 508,11.784,3,0.2,4.2717
3,4,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-ST-10002743,Office Supplies,Storage,SAFCO Boltless Steel Shelving,272.736,3,0.2,-64.7748
4,5,US-2021-141817,2021-01-05,2021-01-12,Standard Class,MB-18085,Mick Brown,Consumer,United States,Philadelphia,...,19143,East,OFF-AR-10003478,Office Supplies,Art,Avery Hi-Liter EverBold Pen Style Fluorescent ...,19.536,3,0.2,4.8840


In [53]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10194 entries, 0 to 10193
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   row_id          10194 non-null  int64         
 1   order_id        10194 non-null  object        
 2   order_date      10194 non-null  datetime64[ns]
 3   ship_date       10194 non-null  datetime64[ns]
 4   ship_mode       10194 non-null  object        
 5   customer_id     10194 non-null  object        
 6   customer_name   10194 non-null  object        
 7   segment         10194 non-null  object        
 8   country_region  10194 non-null  object        
 9   city            10194 non-null  object        
 10  state_province  10194 non-null  object        
 11  postal_code     10194 non-null  object        
 12  region          10194 non-null  object        
 13  product_id      10194 non-null  object        
 14  category        10194 non-null  object        
 15  su

In [62]:
returns_df.columns

Index(['returned', 'order_id'], dtype='object')

### People

In [54]:
people_df.columns = [snake_case(col) for col in people_df.columns]
people_df.head()

,regional_manager,region
0,Sadie Pawthorne,West
1,Chuck Magee,East
2,Roxanne Rodriguez,Central
3,Fred Suzuki,South


### Returns

In [55]:
returns_df.columns = [snake_case(col) for col in returns_df.columns]
returns_df['returned'] = returns_df['returned'].str.replace('Yes', 'Y').replace('No', 'N')
returns_df.head()

,returned,order_id
0,Y,US-2021-100762
4,Y,US-2021-100867
5,Y,US-2021-102652
9,Y,US-2021-103373
10,Y,US-2021-103744


---

## Loading data (MySQL prototype)

- break down the ```Orders``` table into a star schema
- prototype the schema in MySQL

A database, ```supeerstore``` and necessary tables are created in MySQL for prototyping.

### Connecting to MySQL and create ```superstore``` schema

In [56]:
USER = 'root'
HOST = 'localhost'
PASSWORD = '42%Nice69%Evil'
DATABASE = 'superstore'

engine = create_engine(f"mysql+mysqlconnector://{USER}:{PASSWORD}@{HOST}/{DATABASE}")

**Entity Relation Ship Diagram (ERD)**

<img src="https://github.com/user-attachments/assets/73293232-6119-400e-b592-fceb33832016" alt="Image" width="800" height="300">

In [57]:
# try:
# 	with open('load-prototype.sql', 'r') as sql_script:
# 		query = sql_script.read()

# 	with engine.connect() as conn:
# 		# convert the SQL script to SQLAlchemy text obj and run the script
# 		conn.execute(text(query).execution_options(autocommit=True))
# 		print("Tables in superstore db created successfully.")

# except FileNotFoundError:
# 	print('Unable to loacte SQL script.')

# except Exception as error:
# 	print(f"An error occurred: {error}")

### Load data

In [58]:
orders = orders_df[['order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id', 'product_id', 'sales', 'quantity', 'discount', 'profit']]
orders.head()

,order_id,order_date,ship_date,ship_mode,customer_id,product_id,sales,quantity,discount,profit
0,US-2021-103800,2021-01-03,2021-01-07,Standard Class,DP-13000,OFF-PA-10000174,16.448,2,0.2,5.5512
1,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,OFF-BI-10004094,3.540,2,0.8,-5.4870
2,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,OFF-LA-10003223,11.784,3,0.2,4.2717
3,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,OFF-ST-10002743,272.736,3,0.2,-64.7748
4,US-2021-141817,2021-01-05,2021-01-12,Standard Class,MB-18085,OFF-AR-10003478,19.536,3,0.2,4.8840


In [59]:
customers = orders_df[['customer_id', 'customer_name', 'segment', 'country/region', 'city', 'state/province', 'postal_code', 'region']]
customers.head()

KeyError: "['country/region', 'state/province'] not in index"

In [ ]:
products = orders_df[['product_id', 'product_name', 'category', 'sub_category']]
products.head()

KeyError: "['sub_category'] not in index"

In [ ]:
engine = create_engine(f"mysql+mysqlconnector://{USER}:{PASSWORD}@{HOST}/{DATABASE}")
with engine.connect() as conn:
	orders.to_sql('orders', con=conn, if_exists='replace', index=False)
	customers.to_sql('customers', con=conn, if_exists='replace', index=False)